In [ ]:
import tensorflow as tf
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [2]:
seed=42
tf.config.experimental.enable_op_determinism()
tf.random.set_seed(seed)
tf.keras.utils.set_random_seed(seed)
tf.keras.initializers.GlorotNormal(seed)

In [ ]:
target_size=128
batch_size=8

ori_fe_nm_data_path='Data_Preprocessing/data_split_original/training_data/Fe_NM'
imagedatagenerator_path='Data_Preprocessing/data_by_imagedatagenerator/training_data/Fe_NM'

#Original Data location
DataFeNM_classFe = ori_fe_nm_data_path+'/class_fe'
DataFeNM_classNM = ori_fe_nm_data_path+'/class_nm'

#Randomly transformed data location
save_dir_DataFeNM_classFe=imagedatagenerator_path+'/Ferromagnetic'
save_dir_DataFeNM_classNM=imagedatagenerator_path+'/Non_Magnetic'

In [4]:
def transform_data(DataLocation,TransformedDataLocation,batch_size,target_size):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.,
                                                            width_shift_range=0.01,
                                                            height_shift_range=0.01,
                                                            shear_range=0.1,
                                                            zoom_range=0.1,
                                                            horizontal_flip=True,
                                                            vertical_flip=True
                                                                    )
    data_all = datagen.flow_from_directory(DataLocation , 
                                            target_size=(target_size,target_size), #Resizing
                                            batch_size= batch_size,
                                            color_mode='grayscale',
                                            seed=None,
                                            shuffle=True,
                                            class_mode='binary',
                                            save_to_dir=TransformedDataLocation)
    
    #transformed data is generated each time data_all is called 
    total=0
    for image in data_all:
        total +=1
        if total >data_all.num_batches*10: #10 times larger size than original size
            break
    print(f'{len(os.listdir(TransformedDataLocation)) } randomly transformed data located in {TransformedDataLocation}')
    return data_all


In [ ]:
def testing_data(DataLocation,TransformedDataLocation,batch_size,target_size):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.)
    data_all = datagen.flow_from_directory(DataLocation , 
                                            target_size=(target_size,target_size), #Resizing
                                            batch_size= batch_size,
                                            class_mode='binary',
                                            save_to_dir=TransformedDataLocation)
    
    #transformed data is generated each time data_all is called 
    for image in data_all:
            break
    print(f'{len(os.listdir(TransformedDataLocation)) } randomly transformed data located in {TransformedDataLocation}')
    return data_all

*Transform Ferromagnetic DOS Image*

In [6]:
data_all=transform_data(DataFeNM_classFe,save_dir_DataFeNM_classFe,batch_size,target_size)

Found 110 images belonging to 1 classes.


1108 randomly transformed data located in Data_Preprocessing/data_by_imagedatagenerator/Fe_NM/Ferromagnetic


*Transform Non-Magnetic DOS Image*

In [7]:
data_all=transform_data(DataFeNM_classNM,save_dir_DataFeNM_classNM,batch_size,target_size)

Found 119 images belonging to 1 classes.


1198 randomly transformed data located in Data_Preprocessing/data_by_imagedatagenerator/Fe_NM/Non_Magnetic


*Generate Data For Testing Only*

In [24]:
source_dir='Data_Preprocessing/data_all/Fe_NM/original'
save_dir_test_data='Data_Preprocessing/data_all/Fe_NM/testing_data'
testing_data(source_dir,save_dir_test_data,1,target_size)

Found 1 images belonging to 1 classes.
About 1 randomly transformed data located in Data_Preprocessing/data_all/Fe_NM/testing_data


In [8]:
dir_list = os.listdir(imagedatagenerator_path) 
LABELS=['Ferromagnetic', 'Non_Magnetic'] 
data=[]
labels=[]
for label in dir_list:
    if label in LABELS:
        all_image_path=imagedatagenerator_path+'/'+label
        for imagefile in os.listdir(all_image_path):
            imagepath=all_image_path+'/'+imagefile
            image = cv2.imread(imagepath,cv2.IMREAD_GRAYSCALE)

            data.append(image)
            labels.append(label)

In [9]:
data = np.array(data, dtype="float") / 255.0
lb = LabelBinarizer()
labels_ = lb.fit_transform(labels)


In [11]:
train_test_split_dir='Data_Preprocessing/training_data_numpy_format/Fe_NM'
(trainX, valX, trainY, valY) = train_test_split(data, labels_,
	test_size=0.25, stratify=labels, random_state=42)
trainX=trainX.reshape(trainX.shape[0],target_size,target_size,1)
valX=valX.reshape(valX.shape[0],target_size,target_size,1)
np.save(train_test_split_dir+'/trainX',trainX)
np.save(train_test_split_dir+'/trainY',trainY)
np.save(train_test_split_dir+'/valX',valX)
np.save(train_test_split_dir+'/valY',valY)